<a href="https://colab.research.google.com/github/ccarpenterg/LearningPyTorch1.x/blob/master/03_plotting_loss_and_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plotting Loss and Accuracy with PyTorch and Matplotlib

In [0]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import statistics

In [0]:
print('PyTorch version:', torch.__version__)
print('Torchvision version:', torchvision.__version__)

In [0]:
class BasicCNN(nn.module):
    
    
    def __init__(self, num_channels, num_classes):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 32, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0)
        self.pool1 = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(3*3*64, 64, bias=True)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, X):
        x = F.relu(self.conv1(X))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = x.reshape(-1, 3*3*64)
        x = relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [0]:
cuda = torch.device('cuda')


# MNIST images are greyscale (1 channel) and there are 10 digits/classes 
model = BasicCNN(1, 10)
model.to(cuda)

In [0]:
dataset_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.1307], [0.3081])
])

train_set = MNIST('./mnist', train=True, download=True, transform=dataset_transform)
valid_set = MNIST('./mnist', train=False, download=True, transform=dataset_transform)

#checking the dimensions of the training and validation tensors
print(train_set.data.shape)
print(valid_set.data.shape)

In [0]:
train_loader = DataLoader(train_set, batch_size=128, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=512, num_workers=0, shuffle=False)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [0]:
def train(model, loss_fn, optimizer):
    
    model.train()
    train_batch_losses = []
    
    for batch, labels in train_loader:
        
        batch = batch.to(cuda)
        labels = labels.to(cuda)
        
        optimizer.zero_grad()
        y_pred = model(batch)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        train_batch_losses.append(float(loss))
        mean_loss = statistics.mean(train_batch_losses)
        
    return mean_loss